# Photo Data

In [ ]:
from os import listdir

directory = 'Flickr8k/Flicker8k_Dataset'
files = [directory + '/' + name for name in listdir(directory)]

print("Number of images: %d" % len(files))

In [ ]:
import numpy
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16

model = VGG16(include_top=False)
model.summary()

def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    return numpy.expand_dims(x, axis=0)

In [ ]:
from tqdm import tqdm_notebook
from pickle import dump

features = dict()

for file in tqdm_notebook(files):
    tensor = preprocess_input(path_to_tensor(file))
    feature = model.predict(tensor, verbose=0)
    img_id = file.split('.')[0]
    features[img_id] = feature
    
print("Number of features extracted: %d" % len(features))
dump(features, open('features.pkl', 'wb'))